In [2]:
import os
import re
import dotenv
import numpy as np
from numpy import inf
import pandas as pd
from pprint import pprint

In [3]:
# Load environment variables
project_dir = os.path.join(os.path.abspath(''), os.pardir)
dotenv_path = os.path.join(project_dir, '.env')

dotenv.load_dotenv(dotenv_path)

True

In [11]:
df_agg = pd.read_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/daily_measures_features_full.csv', encoding='utf-8')

In [12]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4107 entries, 0 to 4106
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         4107 non-null   object 
 1   Message Text               4107 non-null   object 
 2   Date                       4107 non-null   object 
 3   Impressions                4107 non-null   float64
 4   Comments                   4107 non-null   float64
 5   Likes                      4107 non-null   float64
 6   Shares                     4107 non-null   float64
 7   Other Engagements          4107 non-null   float64
 8   Engagements                4107 non-null   float64
 9   days_since_posted          4107 non-null   float64
 10  fanpage_id                 4107 non-null   float64
 11  has_any_image              4107 non-null   bool   
 12  hashtags                   3281 non-null   object 
 13  has_any_hashtag            4107 non-null   bool 

In [4]:
# # aggregate everything to sum (leistungsdaten) in order to get 1 row per ID
# df_agg = df.groupby(['ID']).agg({
#     'Impressions': 'sum',
#     'Likes': 'sum',
#     'Shares': 'sum',
#     'Comments': 'sum',
#     'Other Engagements': 'sum',
#     'days_since_posted': 'max', # basically as feature more -> better, post acknoledge or good for LI algorithms
#     'Engagements': 'sum',
#     'Date': 'first',
#     'fanpage_id': 'first',
#     'has_any_image': 'first',
#     'hashtags': 'first',
#     'has_any_hashtag': 'first',
#     'hour_posted': 'first',
#     'dow_posted': 'first',
#     'post_len': 'first',
#     'timestamp_posted': 'first',
#     'Dominant_Topic': 'first',
#     'text_clean_and_translated': 'first',
#     'Topic_Perc_Contrib': 'first'
# }).reset_index()

In [13]:
df_agg = df_agg.rename(columns={
    'days_since_posted': 'days_since_last_collected_data',
    'Impressions': 'sum_imps',
    'Engagements': 'sum_engs',
    'Likes': 'sum_likes',
    'Shares': 'sum_shares',
    'Comments': 'sum_comments',
    'Other Engagements': 'sum_engs_other',
    'Dominant_Topic': 'topic_lda',
    'Topic_Perc_Contrib': 'sure_belongs_to_topic',
    'text_clean_and_translated': 'text_posted',
    'post_len': 'length_text_posted'
})

In [14]:
df_agg.iloc[350:355, :]

,ID,Message Text,Date,sum_imps,sum_comments,sum_likes,sum_shares,sum_engs_other,sum_engs,days_since_last_collected_data,...,length_text_posted,timestamp_posted,hm_posted,hour_posted,text_posted,DayOfWeek,log_impressions,log_engagements,topic_lda,sure_belongs_to_topic
350,urn:li:share:7026184646841282561,En kısıtlı alanlarda bile hassas doğrusal hare...,2023-02-02,472.0,0.0,6.0,0.0,4.0,10.0,2.0,...,411,2023-01-31 14:49:24.464,14:49,14,do you need precise linear movements even in t...,3,6.156979,2.302585,3,0.5189
351,urn:li:share:7026194944759377920,"El pasado miércoles 25 de enero, Gabriel River...",2023-02-01,2371.0,0.0,73.0,0.0,141.0,214.0,1.0,...,592,2023-01-31 15:30:15.678,15:30,15,"last wednesday, january 25, gabriel rivera, pr...",2,7.771067,5.365976,0,0.8337
352,urn:li:share:7026195767409192960,With the foundation of the joint venture Cofin...,2023-01-31,11247.0,3.0,214.0,8.0,205.0,430.0,0.0,...,561,2023-01-31 15:33:31.730,15:33,15,with the foundation of the joint venture cofin...,1,9.327857,6.063785,0,0.8357
353,urn:li:share:7026205768064413696,Tem interesse em ingressar em uma empresa líde...,2023-02-20,4418.0,6.0,186.0,30.0,202.0,424.0,20.0,...,529,2023-01-31 16:13:15.834,16:13,16,are you interested in entering a leading techn...,0,8.393442,6.049733,1,0.8389
354,urn:li:share:7026248677476175873,Está no ar mais um episódio do podcast Química...,2023-02-03,179.0,0.0,10.0,1.0,6.0,17.0,3.0,...,653,2023-01-31 19:03:46.329,19:03,19,another episode of the chemistry and innovatio...,4,5.187386,2.833213,0,0.5927


In [15]:
df_agg['timestamp_posted'] = pd.to_datetime(df_agg['timestamp_posted'], format='ISO8601')
df_agg['week_posted'] = df_agg['timestamp_posted'].dt.isocalendar().week
df_agg['weekday_posted'] = df_agg['timestamp_posted'].dt.weekday
df_agg['minute_posted'] = df_agg['timestamp_posted'].dt.minute
df_agg['day_posted'] = df_agg['timestamp_posted'].dt.day

In [16]:
df_agg['topic_lda'] = df_agg['topic_lda'].astype(str)

In [17]:
df_agg = df_agg.set_index('ID')

In [18]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4107 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7091797022185295873
Data columns (total 28 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Message Text                    4107 non-null   object        
 1   Date                            4107 non-null   object        
 2   sum_imps                        4107 non-null   float64       
 3   sum_comments                    4107 non-null   float64       
 4   sum_likes                       4107 non-null   float64       
 5   sum_shares                      4107 non-null   float64       
 6   sum_engs_other                  4107 non-null   float64       
 7   sum_engs                        4107 non-null   float64       
 8   days_since_last_collected_data  4107 non-null   float64       
 9   fanpage_id                      4107 non-null   float64       
 10  has_any_image   

In [20]:
from keybert import KeyBERT

kw_model = KeyBERT()
extract_keywords = lambda doc: kw_model.extract_keywords(doc)

2023-10-14 11:10:46.201069: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 11:10:46.930257: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-10-14 11:10:46.930364: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-1

In [21]:
df_agg['bert_keywords'] = df_agg.text_posted.apply(extract_keywords)

In [22]:
df_agg['bert_keywords'] = df_agg.bert_keywords.apply(lambda x: ','.join([j[0] for j in x]))

In [23]:
df_agg['eng_rate'] = df_agg.sum_engs / df_agg.sum_imps
df_agg['log_eng_rate'] = df_agg.log_engagements / df_agg.log_impressions

In [24]:
# add threshold to exclude new posts and ones with odd data
df_agg_threshold = df_agg[(df_agg.sum_imps >= 50) & (df_agg.days_since_last_collected_data > 0)]

In [26]:
# since sometimes it is impossible to translate hashtags, we will replace the feature with hashtags count
df_agg_threshold.loc[:, 'hashtags_count'] = df_agg_threshold.hashtags.str.split(',').apply(lambda x: len(x) if isinstance(x, list) else 0)

/tmp/ipykernel_1223/2064829808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg_threshold.loc[:, 'hashtags_count'] = df_agg_threshold.hashtags.str.split(',').apply(lambda x: len(x) if isinstance(x, list) else 0)


In [27]:
df_agg_threshold[df_agg_threshold['eng_rate'] >= 1.0]

,Message Text,Date,sum_imps,sum_comments,sum_likes,sum_shares,sum_engs_other,sum_engs,days_since_last_collected_data,fanpage_id,...,topic_lda,sure_belongs_to_topic,week_posted,weekday_posted,minute_posted,day_posted,bert_keywords,eng_rate,log_eng_rate,hashtags_count
ID,,,,,,,,,,,,,,,,,,,,,


In [28]:
X = df_agg_threshold.loc[:, [
    'has_any_image',
    'hashtags_count',
    'has_any_hashtag',
    'week_posted',
    'weekday_posted',
    'day_posted',
    'hour_posted',
    'minute_posted',
    'fanpage_id',
    #'timestamp_posted',
    'length_text_posted',
    'topic_lda',
    'sure_belongs_to_topic',
    'bert_keywords'
]]
y = df_agg_threshold[['eng_rate']]
y_log = df_agg_threshold[['log_eng_rate']]

In [29]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3364 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7090698100771549185
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   has_any_image          3364 non-null   bool   
 1   hashtags_count         3364 non-null   int64  
 2   has_any_hashtag        3364 non-null   bool   
 3   week_posted            3364 non-null   UInt32 
 4   weekday_posted         3364 non-null   int32  
 5   day_posted             3364 non-null   int32  
 6   hour_posted            3364 non-null   int64  
 7   minute_posted          3364 non-null   int32  
 8   fanpage_id             3364 non-null   float64
 9   length_text_posted     3364 non-null   int64  
 10  topic_lda              3364 non-null   object 
 11  sure_belongs_to_topic  3364 non-null   float64
 12  bert_keywords          3364 non-null   object 
dtypes: UInt32(1), bool(2), float64(2), int32(3), int64(3

In [30]:
# One hot encoding for all categorical variables (decision tree needed)
X_dt = X.copy()

In [32]:
import unicodedata as ud
X_dt = pd.concat([
   X_dt,
   X_dt.topic_lda.str.get_dummies().add_prefix(f'lda_topic_'),
    X_dt.topic_lda.str.get_dummies().add_prefix(f'lda_topic_'),
    #'week_posted', 'day_posted', 'hour_posted', 'minute_posted', 'weekday_posted'
   pd.get_dummies(X_dt.bert_keywords.str.split(',').explode()).groupby(level=0).sum()
], axis=1).drop(columns=['topic_lda', 'bert_keywords'])

In [33]:
X_dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3364 entries, urn:li:share:7015109880122384384 to urn:li:ugcPost:7090698100771549185
Columns: 5115 entries, has_any_image to 新春快乐
dtypes: UInt32(1), bool(2), float64(2), int32(3), int64(5107)
memory usage: 131.2+ MB


In [34]:
X_dt

,has_any_image,hashtags_count,has_any_hashtag,week_posted,weekday_posted,day_posted,hour_posted,minute_posted,fanpage_id,length_text_posted,...,zetuvit,zhaoting,zierath,zink,znozsvi6qc,zodiac,zone,zones,zum,新春快乐
ID,,,,,,,,,,,,,,,,,,,,,
urn:li:share:7015109880122384384,False,6,True,52,6,1,1,22,154517.0,213,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015180072244367361,True,3,True,52,6,1,6,1,50.0,266,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015210047886524416,True,0,False,52,6,1,8,0,118.0,330,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015240414274158592,True,0,False,52,6,1,10,0,2165.0,189,...,0,0,0,0,0,0,0,0,0,0
urn:li:share:7015572647832948736,True,0,False,1,0,2,8,1,33166.0,842,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
urn:li:ugcPost:7089944977115340800,False,3,True,30,2,26,14,30,7958.0,426,...,0,0,0,0,0,0,0,0,0,0
urn:li:ugcPost:7090219491409731584,False,1,True,30,3,27,8,40,118.0,826,...,0,0,0,0,0,0,0,0,0,0
urn:li:ugcPost:7090288923028926464,False,1,True,30,3,27,13,16,118.0,787,...,0,0,0,0,0,0,0,0,0,0


In [35]:
y.describe()

,eng_rate
count,3364.000000
mean,0.067571
std,0.073429
min,0.000000
25%,0.027230
50%,0.043367
75%,0.076765
max,0.679137


In [36]:
X.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/X_catboost.csv', encoding='utf-8', index=True)
X_dt.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/X_decision_trees.csv', encoding='utf-8', index=True)
y.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/y_both.csv', encoding='utf-8', index=True)
y_log.to_csv('/home/jovyan/git_projects/machine-learning-project/data/processed/y_log_both.csv', encoding='utf-8', index=True)